<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/DEEPSEEK_AGENT_CREWAI_AOC_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Apr  9 22:06:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   40C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install --upgrade crewai langchain -q
!pip install --upgrade colab-env --quiet
!pip install --upgrade langchain_community -q
!pip install --upgrade faiss-cpu -q
!pip install --upgrade crewai-tools -q
!pip install 'crewai[tools]' -q
!pip install transformers -U -q

In [ ]:
import warnings

warnings.filterwarnings("ignore", message="You seem to be using the pipelines sequentially on GPU")

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from langchain_community.llms import HuggingFaceHub

In [ ]:
import colab_env
import os

In [ ]:
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from litellm import completion  # Import litellm directly

load_dotenv()

# 1. Token Management (Assuming DeepSeek API key is in HUGGINGFACE_API_KEY environment variable)
access_token_write = os.getenv("HUGGINGFACE_API_KEY")

# 2. Initialize LLM (Using DeepSeek-V3)
model_name = "deepseek-ai/DeepSeek-V3"

# 3. Define Agents (Modify agent behavior to use litellm.completion)
def agent_run_with_litellm(self, task_description):
    """
    This function replaces the default agent run behavior to use litellm.completion.
    """
    messages = [{"role": "user", "content": task_description}]
    response = completion(model=model_name, messages=messages, api_key=access_token_write)
    return response.choices[0].message.content

# Monkey patch the agent's run method
Agent.run = agent_run_with_litellm

# 4. Define Agents (AIRLINE OPERATION CONTROL)
scheduler = Agent(
    role='Flight Scheduler',
    goal='Optimize flight schedules and manage aircraft assignments.',
    backstory="You are an experienced flight scheduler responsible for creating efficient flight plans and minimizing delays.",
    allow_delegation=False,
    verbose=True,
)

crew_dispatcher = Agent(
    role='Crew Dispatcher',
    goal='Assign crew members to flights based on qualifications and regulations.',
    backstory="You are a skilled crew dispatcher ensuring flights are staffed with qualified personnel while adhering to regulations.",
    allow_delegation=False,
    verbose=True,
)

disruption_handler = Agent(
    role='Disruption Handler',
    goal='Manage flight disruptions, re-routing aircraft and passengers as needed.',
    backstory="You are a resourceful disruption handler, adept at finding solutions to minimize the impact of unexpected events.",
    allow_delegation=False,
    verbose=True,
)

passenger_coordinator = Agent(
    role='Passenger Coordinator',
    goal='Assist passengers with travel arrangements, rebooking, and special needs.',
    backstory="You are a dedicated passenger coordinator, ensuring a smooth and positive travel experience for all passengers.",
    allow_delegation=False,
    verbose=True,
)

maintenance_coordinator = Agent(
    role='Maintenance Coordinator',
    goal='Schedule aircraft maintenance and manage repair resources.',
    backstory="You are a skilled maintenance coordinator, ensuring aircraft are airworthy and maintained according to regulations.",
    allow_delegation=False,
    verbose=True,
)

# 5. Define Tasks (AIRLINE OPERATION CONTROL)
schedule_optimization_task = Task(
    description="Optimize the flight schedule for the next 24 hours, considering aircraft availability, crew schedules, and expected passenger demand. Minimize delays and maximize aircraft utilization.",
    agent=scheduler,
    expected_output="An optimized flight schedule for the next 24 hours with aircraft and crew assignments.",
)

crew_assignment_task = Task(
    description="Assign crew members to the flights in the optimized schedule, ensuring all roles are filled by qualified personnel and regulatory requirements are met.",
    agent=crew_dispatcher,
    expected_output="A crew assignment plan for all flights in the schedule.",
)

disruption_management_task = Task(
    description="A severe weather event is causing delays at a major hub airport. Develop a plan to minimize disruption to passengers and operations, including re-routing aircraft and crew.",
    agent=disruption_handler,
    expected_output="A disruption management plan outlining actions to mitigate the impact of the weather event.",
)

passenger_assistance_task = Task(
    description="A flight is delayed, and passengers need to be rebooked on alternative flights or provided with accommodations. Coordinate with the scheduler and crew dispatcher to minimize passenger inconvenience.",
    agent=passenger_coordinator,
    expected_output="A passenger assistance plan for the delayed flight, including rebooking options and accommodation arrangements.",
)

maintenance_scheduling_task = Task(
    description="An aircraft requires routine maintenance. Schedule the maintenance during a non-operational window to minimize disruption to the flight schedule.",
    agent=maintenance_coordinator,
    expected_output="A maintenance schedule for the aircraft, ensuring minimal impact on flight operations.",
)

# 6. Create and Run the Crew
crew = Crew(
    agents=[scheduler, crew_dispatcher, disruption_handler, passenger_coordinator, maintenance_coordinator],
    tasks=[schedule_optimization_task, crew_assignment_task, disruption_management_task, passenger_assistance_task, maintenance_scheduling_task],
    process=Process.sequential
)

try:
    result = crew.kickoff()
    print("Crew's Output:")
    print(result)

except Exception as e:
    print(f"An error occurred during crew execution: {e}")

# Agent: Flight Scheduler
## Task: Optimize the flight schedule for the next 24 hours, considering aircraft availability, crew schedules, and expected passenger demand. Minimize delays and maximize aircraft utilization.


# Agent: Flight Scheduler
## Final Answer: 
**Optimized Flight Schedule for the Next 24 Hours**

**Aircraft Availability:**
- Total Aircraft: 10
- Aircraft Status:
  - 5 Aircraft available for the full day (A1, A2, A3, A4, A5)
  - 2 Aircraft on maintenance (A6, A7)
  - 3 Aircraft returning from regional flights (A8, A9, A10) with arrival at different times.

**Crew Schedules:**
- Total Crew Members: 20
- Crew Assignments:
  - 5 pilots and 5 cabin crew members available for A1, A2, A3, A4, and A5.
  - 4 pilots and 4 cabin crew members are available for A8, A9, and A10 upon their return.

**Expected Passenger Demand:**
- Peak Hours: 08:00 AM - 10:00 AM; 06:00 PM - 08:00 PM
- Routes with High Demand:
  1. New York (JFK) to Los Angeles (LAX)
  2. Chicago (ORD) to Miami (M